# Interactive Data validation and preparation file.

In [80]:
# Todo: Refactor all translations process. Script to create dictionanries, dump to json, fuction to create english, french, greek data files.

In [81]:
import pandas as pd
import numpy as np

from parse_nace_codes import parse_nace_codes

# Display numpy numbers with two decimal places
np.set_printoptions(precision=2)

# Display pandas numbers with two decimal places
pd.set_option("display.float_format", lambda x: "%.2f" % x)

In [82]:
# Files location
mis_file_path = "../data/rrf_smrt_manuf_mis_granular_data.xlsx"
official_data_file_path = "../data/rrf_smrt_manuf_funded_proposals_official_data.xlsx"

## Read Management Information System (MIS) data

In [83]:
# Load MIS excel file
with pd.ExcelFile(mis_file_path, engine="openpyxl") as mis_excel_file:
    # Get sheet names list
    sheet_names = mis_excel_file.sheet_names

    # Get sheets as dicts
    mis_dfs = pd.read_excel(
        mis_file_path,
        sheet_name=sheet_names,
        engine="openpyxl",
    )

In [84]:
# Convert sheets dictionaries to dataframes
mis_proposals_df = mis_dfs[sheet_names[0]]
mis_sectors_df = mis_dfs[sheet_names[1]]

In [85]:
mis_proposals_df.columns

Index(['Κωδικός Πρότασης', 'ΦΑΣΗ ΠΡΟΤΑΣΗΣ', 'Κατάσταση Αιτήματος',
       'Τρέχουσα Κατάσταση Πρότασης', 'Επωνυμία Φορέα', 'Μέγεθος',
       'Είδος Επιχείρησης', 'Περιφέρεια (unique)', 'ΠΕ (unique)',
       'Συνολικός Προϋπολογισμός', 'Επιχορηγούμενος Προϋπολογισμός',
       'Αιτούμενη Επιχορήγηση (ΔΔ)'],
      dtype='object')

In [86]:
# remove 'Επωνυμία Φορέα' column
mis_proposals_df = mis_proposals_df.drop(columns=["Επωνυμία Φορέα"])

In [87]:
mis_proposals_df["Κατάσταση Αιτήματος"].unique()

array(['Απορριφθείσα', 'Εγκεκριμένη',
       'Έχει υποβληθεί - Δεν έχει παραληφθεί'], dtype=object)

In [88]:
mis_approved_proposals = mis_proposals_df[
    mis_proposals_df["Κατάσταση Αιτήματος"] == "Εγκεκριμένη"
]

In [89]:
mis_approved_codes_list = list(mis_approved_proposals["Κωδικός Πρότασης"].values)
# strip all whitespaces
mis_approved_codes_list = [x.strip() for x in mis_approved_codes_list]

## Read official published data

In [90]:
official_data_df = pd.read_excel(official_data_file_path, engine="openpyxl")

In [91]:
official_projects_codes_list = list(official_data_df["Κωδικός Πρότασης"].values)
# strip all whitespaces
official_projects_codes_list = [x.strip() for x in official_projects_codes_list]

### 1. Data validation and quality

In [92]:
# Test if both lists have the same length
len(official_projects_codes_list) == len(mis_approved_codes_list)

True

In [93]:
# Compare items in the two lists
# TODO: refactor using sets
items_not_in_mis = [
    item for item in official_projects_codes_list if item not in mis_approved_codes_list
]
items_not_in_mis

[]

In [94]:
# TODO: Test if total approved budget is the same in both datasets

In [95]:
# Test if all proposals have been reviewed and submitted in the MIS
mis_proposals_df["Κατάσταση Αιτήματος"].unique()

array(['Απορριφθείσα', 'Εγκεκριμένη',
       'Έχει υποβληθεί - Δεν έχει παραληφθεί'], dtype=object)

In [96]:
non_submitted_df = mis_proposals_df[
    mis_proposals_df["Κατάσταση Αιτήματος"] == "Έχει υποβληθεί - Δεν έχει παραληφθεί"
]  # Rejected proposal.
non_submitted_df

,Κωδικός Πρότασης,ΦΑΣΗ ΠΡΟΤΑΣΗΣ,Κατάσταση Αιτήματος,Τρέχουσα Κατάσταση Πρότασης,Μέγεθος,Είδος Επιχείρησης,Περιφέρεια (unique),ΠΕ (unique),Συνολικός Προϋπολογισμός,Επιχορηγούμενος Προϋπολογισμός,Αιτούμενη Επιχορήγηση (ΔΔ)
127,ΕΞΜΠ-0544761,Γνωμοδοτική Ένταξης,Έχει υποβληθεί - Δεν έχει παραληφθεί,Απορριφθείσα,Πολύ Μικρή,Υφιστάμενη Επιχείρηση,Κρήτης,ΛΑΣΙΘΙΟΥ,276605.80,276605.80,165963.48


### 2. Drop and rename columns

In [97]:
df = mis_approved_proposals

In [98]:
cols_to_drop = ["ΦΑΣΗ ΠΡΟΤΑΣΗΣ", "Κατάσταση Αιτήματος", "Τρέχουσα Κατάσταση Πρότασης"]
df = df.drop(columns=cols_to_drop)

In [99]:
df.columns

Index(['Κωδικός Πρότασης', 'Μέγεθος', 'Είδος Επιχείρησης',
       'Περιφέρεια (unique)', 'ΠΕ (unique)', 'Συνολικός Προϋπολογισμός',
       'Επιχορηγούμενος Προϋπολογισμός', 'Αιτούμενη Επιχορήγηση (ΔΔ)'],
      dtype='object')

In [100]:
# rename, translate column names
df.columns = [
    "project_code",
    "size",
    "incorporation_status",
    "region",
    "regional_unit",
    "investment_budget",
    "eligible_budget",
    "public_funding",
]

In [101]:
df.project_code.unique()[:3]

array(['ΕΞΜΠ-0543351                  ', 'ΕΞΜΠ-0543354                  ',
       'ΕΞΜΠ-0543368                  '], dtype=object)

In [102]:
# strip whitespaces from df["project_code"]
df["project_code"] = df["project_code"].str.strip()

In [103]:
mis_sectors_df.columns

Index(['Κωδικός Πρότασης', 'ΚΑΔ', '4ψήφιος ΚΑΔ - Κωδικός', 'Περιγραφή ΚΑΔ',
       'Κύριος ΚΑΔ', 'ΚΑΔ Επένδυσης?'],
      dtype='object')

In [104]:
# keep only necessary columns
mis_sectors_df.columns = [
    "project_code",
    "nace_8_digits",
    "nace_4_digits",
    "description_nace_8_digits",
    "main_nace",
    "investment_nace",
]

In [105]:
mis_sectors_df["investment_nace"].unique()

array(['NAI', 'OXI'], dtype=object)

In [106]:
mis_sectors_df["project_code"].unique()[:3]

array(['ΕΞΜΠ-0543351                  ', 'ΕΞΜΠ-0543354                  ',
       'ΕΞΜΠ-0543368                  '], dtype=object)

In [107]:
# strip whitespaces from mis_sectors_df["project_code"]
mis_sectors_df["project_code"] = mis_sectors_df["project_code"].str.strip()

In [108]:
mis_sectors_df["project_code"].unique()[:3]

array(['ΕΞΜΠ-0543351', 'ΕΞΜΠ-0543354', 'ΕΞΜΠ-0543368'], dtype=object)

## 3. Focus on the approved proposals for investment

### 3.1. Keep only investments' sectors

In [109]:
investment_sectors = mis_sectors_df[mis_sectors_df["investment_nace"] == "NAI"]

In [110]:
len(investment_sectors)

351

In [111]:
other_sectors = mis_sectors_df[mis_sectors_df["investment_nace"] == "OXI"]
len(other_sectors)

1198

In [112]:
len(mis_sectors_df["nace_4_digits"].unique())

307

In [113]:
investment_sectors.head(10)

,project_code,nace_8_digits,nace_4_digits,description_nace_8_digits,main_nace,investment_nace
0,ΕΞΜΠ-0543351,11.05.10,11.05,"Παραγωγή μπίρας, εκτός από κατάλοιπα ζυθοποιίας",NAI,NAI
3,ΕΞΜΠ-0543354,25.62.10,25.62,Υπηρεσίες τόρνευσης μεταλλικών μερών,OXI,NAI
4,ΕΞΜΠ-0543354,25.62.20,25.62,Άλλες υπηρεσίες μεταλλοτεχνίας,OXI,NAI
5,ΕΞΜΠ-0543354,28.11.41,28.11,Κατασκευή μερών κινητήρων εσωτερικής καύσης στ...,NAI,NAI
23,ΕΞΜΠ-0543368,31.02.10,31.02,Κατασκευή επίπλων κουζίνας,OXI,NAI
24,ΕΞΜΠ-0543368,31.09.12,31.09,Κατασκευή ξύλινων επίπλων που χρησιμοποιούνται...,NAI,NAI
25,ΕΞΜΠ-0543368,31.09.13,31.09,Κατασκευή ξύλινων επίπλων π.δ.κ.α.,OXI,NAI
26,ΕΞΜΠ-0543368,31.09.92.01,31.09,Υπηρεσίες αποκατάστασης και ανακαίνισης επίπλων,OXI,NAI
41,ΕΞΜΠ-0543374,25.62.1,25.62,Υπηρεσίες τόρνευσης μεταλλικών μερών,OXI,NAI
42,ΕΞΜΠ-0543374,25.99,25.99,Κατασκευή άλλων μεταλλικών προϊόντων π.δ.κ.α.,OXI,NAI


The description corresponds to the 6 or 8 digit NACE codes.

### 3.2 Merge MIS approved proposals data with sectors data

In [114]:
# merge proposals and sectors dataframes on 'Κωδικός Πρότασης'
merged_sectors_df = pd.merge(df, investment_sectors, on="project_code")

In [115]:
len(merged_sectors_df)  # in 6 digits NACE codes

351

In [116]:
merged_sectors_df["project_code"].nunique()

151

#### Note: There are 351 sector rows for 151 projects. This is because many projects are investing in multiple sectors.

### 3.3 Convert NACE codes to two digits

In [117]:
# test if there are codes below 10, that might miss the first zero
sorted(merged_sectors_df["nace_4_digits"].unique())[:5]

[np.float64(10.13),
 np.float64(10.31),
 np.float64(10.39),
 np.float64(10.41),
 np.float64(10.51)]

In [118]:
# convert nace_4_digits and nace_8_digits to string. TODO: Do it in one line.
merged_sectors_df["nace_4_digits"] = merged_sectors_df["nace_4_digits"].apply(str)
merged_sectors_df["nace_8_digits"] = merged_sectors_df["nace_8_digits"].apply(str)

In [119]:
# get first two digits of the nace_4_digits code and save it in a new column
merged_sectors_df["nace_2_digits"] = merged_sectors_df["nace_4_digits"].apply(
    lambda x: x[:2]
)

In [120]:
# data validation test
merged_sectors_df.investment_nace.unique()

array(['NAI'], dtype=object)

In [121]:
merged_sectors_df.columns

Index(['project_code', 'size', 'incorporation_status', 'region',
       'regional_unit', 'investment_budget', 'eligible_budget',
       'public_funding', 'nace_8_digits', 'nace_4_digits',
       'description_nace_8_digits', 'main_nace', 'investment_nace',
       'nace_2_digits'],
      dtype='object')

In [122]:
# keep necessary columns for two digit sectoral analysis
large_sectors_df = merged_sectors_df[
    [
        "project_code",
        "size",
        "incorporation_status",
        "region",
        "investment_budget",
        "public_funding",
        "nace_2_digits",
    ]
]

In [123]:
large_sectors_df.head(2)

,project_code,size,incorporation_status,region,investment_budget,public_funding,nace_2_digits
0,ΕΞΜΠ-0543351,Μικρή,Υφιστάμενη Επιχείρηση,Ιονίων Νήσων,482832.00,260504.28,11
1,ΕΞΜΠ-0543354,Μεσαία,Υφιστάμενη Επιχείρηση,Αττικής,729388.06,255285.82,25


In [124]:
# Number of sectors
large_sectors_df["nace_2_digits"].nunique()

24


### 3.4 Add two digits sector descriptions

In [125]:
# Use the NACE codes dictionary to get the sector descriptions
nace_codes_file = "../data/EconActiv_nace_2_gr.xls"

In [126]:
# Use custom function
nace_2_digits_descriptions = parse_nace_codes(nace_codes_file)

In [127]:
large_sectors_df = large_sectors_df.assign(
    nace_2_digits_description=large_sectors_df["nace_2_digits"].map(
        nace_2_digits_descriptions
    )
)

In [128]:
large_sectors_df.head(2)

,project_code,size,incorporation_status,region,investment_budget,public_funding,nace_2_digits,nace_2_digits_description
0,ΕΞΜΠ-0543351,Μικρή,Υφιστάμενη Επιχείρηση,Ιονίων Νήσων,482832.00,260504.28,11,Ποτοποιία
1,ΕΞΜΠ-0543354,Μεσαία,Υφιστάμενη Επιχείρηση,Αττικής,729388.06,255285.82,25,"Κατασκευή μεταλλικών προϊόντων, με εξαίρεση τα..."


### 3.5 Add letters for General sector description

In [129]:
# Groupes sectors by general sector description
general_sectors = {
    "C: Manufacturing": list(range(10, 34)),
    "E: Water supply, sewerage, waste management and remediation": list(range(36, 40)),
    "H: Transportation and Storage": list(range(49, 54)),
    "M: Legal, accounting, management, architecture, engineering, technical testing and analysis activities": list(
        range(69, 72)
    ),
    "N: Administrative and support service activities": list(range(77, 83)),
}


# Function to map the two-digit codes to the general sector descriptions
def map_classification(nace_code):
    for sector, codes in general_sectors.items():
        if int(nace_code) in codes:
            return sector
    return "Other"

In [130]:
# Apply the function to create a new column in totals_df
large_sectors_df["sector"] = large_sectors_df["nace_2_digits"].apply(map_classification)

In [131]:
large_sectors_df.head(5)

,project_code,size,incorporation_status,region,investment_budget,public_funding,nace_2_digits,nace_2_digits_description,sector
0,ΕΞΜΠ-0543351,Μικρή,Υφιστάμενη Επιχείρηση,Ιονίων Νήσων,482832.00,260504.28,11,Ποτοποιία,C: Manufacturing
1,ΕΞΜΠ-0543354,Μεσαία,Υφιστάμενη Επιχείρηση,Αττικής,729388.06,255285.82,25,"Κατασκευή μεταλλικών προϊόντων, με εξαίρεση τα...",C: Manufacturing
2,ΕΞΜΠ-0543354,Μεσαία,Υφιστάμενη Επιχείρηση,Αττικής,729388.06,255285.82,25,"Κατασκευή μεταλλικών προϊόντων, με εξαίρεση τα...",C: Manufacturing
3,ΕΞΜΠ-0543354,Μεσαία,Υφιστάμενη Επιχείρηση,Αττικής,729388.06,255285.82,28,Κατασκευή μηχανημάτων και ειδών εξοπλισμού π.δ...,C: Manufacturing
4,ΕΞΜΠ-0543368,Πολύ Μικρή,Υφιστάμενη Επιχείρηση,Κεντρικής Μακεδονίας,359000.00,251300.00,31,Κατασκευή επίπλων,C: Manufacturing


In [132]:
# split sector column to two columns. One for the sector letter and one for the sector description
large_sectors_df["sector_letter"] = large_sectors_df["sector"].str.split(":").str[0]
large_sectors_df["sector_description"] = (
    large_sectors_df["sector"].str.split(":").str[1]
)
# drop sector column
large_sectors_df = large_sectors_df.drop(columns=["sector"])

In [133]:
# Replace unique values with new project_id values starting from p_1 to p{len(df)}
# Dictionary to map old project codes to new project IDs
project_id_mapping = {
    old_code: f"p_{i+1}"
    for i, old_code in enumerate(large_sectors_df["project_code"].unique())
}

In [134]:
project_id_mapping

{'ΕΞΜΠ-0543351': 'p_1',
 'ΕΞΜΠ-0543354': 'p_2',
 'ΕΞΜΠ-0543368': 'p_3',
 'ΕΞΜΠ-0543374': 'p_4',
 'ΕΞΜΠ-0543375': 'p_5',
 'ΕΞΜΠ-0543386': 'p_6',
 'ΕΞΜΠ-0543387': 'p_7',
 'ΕΞΜΠ-0543391': 'p_8',
 'ΕΞΜΠ-0543401': 'p_9',
 'ΕΞΜΠ-0543404': 'p_10',
 'ΕΞΜΠ-0543414': 'p_11',
 'ΕΞΜΠ-0543439': 'p_12',
 'ΕΞΜΠ-0543458': 'p_13',
 'ΕΞΜΠ-0543464': 'p_14',
 'ΕΞΜΠ-0543493': 'p_15',
 'ΕΞΜΠ-0543512': 'p_16',
 'ΕΞΜΠ-0543553': 'p_17',
 'ΕΞΜΠ-0543574': 'p_18',
 'ΕΞΜΠ-0543657': 'p_19',
 'ΕΞΜΠ-0543780': 'p_20',
 'ΕΞΜΠ-0543816': 'p_21',
 'ΕΞΜΠ-0543850': 'p_22',
 'ΕΞΜΠ-0543893': 'p_23',
 'ΕΞΜΠ-0543963': 'p_24',
 'ΕΞΜΠ-0543972': 'p_25',
 'ΕΞΜΠ-0544008': 'p_26',
 'ΕΞΜΠ-0544036': 'p_27',
 'ΕΞΜΠ-0544053': 'p_28',
 'ΕΞΜΠ-0544054': 'p_29',
 'ΕΞΜΠ-0544149': 'p_30',
 'ΕΞΜΠ-0544521': 'p_31',
 'ΕΞΜΠ-0544718': 'p_32',
 'ΕΞΜΠ-0544732': 'p_33',
 'ΕΞΜΠ-0544746': 'p_34',
 'ΕΞΜΠ-0544774': 'p_35',
 'ΕΞΜΠ-0544782': 'p_36',
 'ΕΞΜΠ-0544802': 'p_37',
 'ΕΞΜΠ-0544826': 'p_38',
 'ΕΞΜΠ-0544838': 'p_39',
 'ΕΞΜΠ-0544841': 'p_40',
 'ΕΞΜΠ-05

In [135]:
# Replace old project codes with new project IDs
large_sectors_df["project_code"] = large_sectors_df["project_code"].map(
    project_id_mapping
)

In [136]:
len(large_sectors_df)

351

In [137]:
large_sectors_df.project_code.nunique()

151

### 3.6 Translate all column values to English

In [138]:
df = large_sectors_df.copy()

In [139]:
df.nace_2_digits_description.unique()

array(['Ποτοποιία',
       'Κατασκευή μεταλλικών προϊόντων, με εξαίρεση τα μηχανήματα και τα είδη εξοπλισμού',
       'Κατασκευή μηχανημάτων και ειδών εξοπλισμού π.δ.κ.α.',
       'Κατασκευή επίπλων',
       'Επισκευή και εγκατάσταση μηχανημάτων και εξοπλισμού',
       'Συλλογή, επεξεργασία και διάθεση αποβλήτων? ανάκτηση υλικών',
       'Βιομηχανία τροφίμων', 'Παραγωγή χημικών ουσιών και προϊόντων',
       'Κατασκευή ειδών ένδυσης',
       'Εκτυπώσεις και αναπαραγωγή προεγγεγραμμένων μέσων',
       'Παραγωγή κλωστοϋφαντουργικών υλών',
       'Άλλες μεταποιητικές δραστηριότητες',
       'Κατασκευή προϊόντων από ελαστικό (καουτσούκ) και πλαστικές ύλες',
       'Αρχιτεκτονικές δραστηριότητες και δραστηριότητες μηχανικών? τεχνικές δοκιμές και αναλύσεις',
       'Χαρτοποιία και κατασκευή χάρτινων προϊόντων',
       'Αποθήκευση και υποστηρικτικές προς τη μεταφορά δραστηριότητες',
       'Βιομηχανία ξύλου και κατασκευή προϊόντων από ξύλο και φελλό, εκτός από έπιπλα? κατασκευή ειδών καλαθοποι

In [140]:
df.nace_2_digits_description.nunique()

24

In [141]:
# Get unique values from the 'nace_2_digits' column
unique_nace_2_digits = df["nace_2_digits"].unique()

# Dictionary with all possible descriptions
NACE_2_DIGITS_DESCRIPTIONS_EN = {
    "10": "Manufacture of food products",
    "11": "Manufacture of beverages",
    "12": "Manufacture of tobacco products",
    "13": "Manufacture of textiles",
    "14": "Manufacture of wearing apparel",
    "15": "Manufacture of leather and related products",
    "16": "Manufacture of wood and of products of wood and cork, except furniture; manufacture of articles of straw and plaiting materials",
    "17": "Manufacture of paper and paper products",
    "18": "Printing and reproduction of recorded media",
    "19": "Manufacture of coke and refined petroleum products",
    "20": "Manufacture of chemicals and chemical products",
    "21": "Manufacture of basic pharmaceutical products and pharmaceutical preparations",
    "22": "Manufacture of rubber and plastic products",
    "23": "Manufacture of other non-metallic mineral products",
    "24": "Manufacture of basic metals",
    "25": "Manufacture of fabricated metal products, except machinery and equipment",
    "26": "Manufacture of computer, electronic and optical products",
    "27": "Manufacture of electrical equipment",
    "28": "Manufacture of machinery and equipment n.e.c.",
    "29": "Manufacture of motor vehicles, trailers and semi-trailers",
    "30": "Manufacture of other transport equipment",
    "31": "Manufacture of furniture",
    "32": "Other manufacturing",
    "33": "Repair and installation of machinery and equipment",
    "36": "Water collection, treatment and supply",
    "37": "Sewerage",
    "38": "Waste collection, treatment and disposal activities; materials recovery",
    "39": "Remediation activities and other waste management services",
    "49": "Land transport and transport via pipelines",
    "50": "Water transport",
    "51": "Air transport",
    "52": "Warehousing and support activities for transportation",
    "53": "Postal and courier activities",
    "69": "Legal and accounting activities",
    "70": "Activities of head offices; management consultancy activities",
    "71": "Architectural and engineering activities; technical testing and analysis",
    "72": "Scientific research and development",
    "77": "Rental and leasing activities",
    "78": "Employment activities",
    "79": "Travel agency, tour operator and other reservation service and related activities",
    "80": "Security and investigation activities",
    "81": "Services to buildings and landscape activities",
    "82": "Office administrative, office support and other business support activities",
}

# Filter dictionary with activities present in the data
filtered_nace_2_digits_descriptions_en = {
    k: v for k, v in NACE_2_DIGITS_DESCRIPTIONS_EN.items() if k in unique_nace_2_digits
}
filtered_nace_2_digits_descriptions_en

{'10': 'Manufacture of food products',
 '11': 'Manufacture of beverages',
 '13': 'Manufacture of textiles',
 '14': 'Manufacture of wearing apparel',
 '16': 'Manufacture of wood and of products of wood and cork, except furniture; manufacture of articles of straw and plaiting materials',
 '17': 'Manufacture of paper and paper products',
 '18': 'Printing and reproduction of recorded media',
 '20': 'Manufacture of chemicals and chemical products',
 '21': 'Manufacture of basic pharmaceutical products and pharmaceutical preparations',
 '22': 'Manufacture of rubber and plastic products',
 '23': 'Manufacture of other non-metallic mineral products',
 '24': 'Manufacture of basic metals',
 '25': 'Manufacture of fabricated metal products, except machinery and equipment',
 '26': 'Manufacture of computer, electronic and optical products',
 '27': 'Manufacture of electrical equipment',
 '28': 'Manufacture of machinery and equipment n.e.c.',
 '29': 'Manufacture of motor vehicles, trailers and semi-trai

In [142]:
# Test for N of unique values
len(filtered_nace_2_digits_descriptions_en)

24

In [143]:
# Todo: test for mistranslations

In [144]:
# Map existing values in the `nace_2_digits` column to English
df["nace_2_digits_description"] = df["nace_2_digits"].replace(
    filtered_nace_2_digits_descriptions_en
)

# Display the updated DataFrame
df.head()

,project_code,size,incorporation_status,region,investment_budget,public_funding,nace_2_digits,nace_2_digits_description,sector_letter,sector_description
0,p_1,Μικρή,Υφιστάμενη Επιχείρηση,Ιονίων Νήσων,482832.00,260504.28,11,Manufacture of beverages,C,Manufacturing
1,p_2,Μεσαία,Υφιστάμενη Επιχείρηση,Αττικής,729388.06,255285.82,25,"Manufacture of fabricated metal products, exce...",C,Manufacturing
2,p_2,Μεσαία,Υφιστάμενη Επιχείρηση,Αττικής,729388.06,255285.82,25,"Manufacture of fabricated metal products, exce...",C,Manufacturing
3,p_2,Μεσαία,Υφιστάμενη Επιχείρηση,Αττικής,729388.06,255285.82,28,Manufacture of machinery and equipment n.e.c.,C,Manufacturing
4,p_3,Πολύ Μικρή,Υφιστάμενη Επιχείρηση,Κεντρικής Μακεδονίας,359000.00,251300.00,31,Manufacture of furniture,C,Manufacturing


In [145]:
# Translate the 'size' column
size_values = df["size"].unique()
size_values

array(['Μικρή', 'Μεσαία', 'Πολύ Μικρή'], dtype=object)

In [146]:
# Use the size_values array as keys to create dictionary
SIZE_TRANSLATIONS = {
    "Μικρή": "Small",
    "Μεσαία": "Medium",
    "Πολύ Μικρή": "Very small",
}

# Map existing values in the `size` column to English
# df['size'] = df['size'].map(SIZE_TRANSLATIONS)
df["size"] = df["size"].replace(SIZE_TRANSLATIONS)

In [147]:
df["size"].unique()

array(['Small', 'Medium', 'Very small'], dtype=object)

In [148]:
# Translate the 'incorporation_status' column
df["incorporation_status"].unique()

array(['Υφιστάμενη Επιχείρηση', 'Νέα Επιχείρηση'], dtype=object)

In [149]:
INCORP_TRANSLATIONS = {
    "Υφιστάμενη Επιχείρηση": "Existing Business",
    "Νέα Επιχείρηση": "New Business",
}

# Map existing values in the `incorporation_status` column to English
df["incorporation_status"] = df["incorporation_status"].replace(INCORP_TRANSLATIONS)

In [150]:
# Translate the 'region' column values
df["region"].unique()

array(['Ιονίων Νήσων', 'Αττικής', 'Κεντρικής Μακεδονίας', 'Θεσσαλίας',
       'Κρήτης', 'Στερεάς Ελλάδας', 'Ηπείρου', 'Πελοποννήσου',
       'Δυτικής Ελλάδας', 'Ανατολικής Μακεδονίας και Θράκης',
       'Δυτικής Μακεδονίας', 'Νοτίου Αιγαίου', 'Βορείου Αιγαίου'],
      dtype=object)

In [151]:
REGION_TRANSLATIONS = {
    "Ιονίων Νήσων": "Ionian Islands",
    "Αττικής": "Attica",
    "Κεντρικής Μακεδονίας": "Central Macedonia",
    "Θεσσαλίας": "Thessaly",
    "Κρήτης": "Crete",
    "Στερεάς Ελλάδας": "Central Greece",
    "Ηπείρου": "Epirus",
    "Πελοποννήσου": "Peloponnese",
    "Δυτικής Ελλάδας": "Western Greece",
    "Ανατολικής Μακεδονίας και Θράκης": "Eastern Macedonia and Thrace",
    "Δυτικής Μακεδονίας": "Western Macedonia",
    "Νοτίου Αιγαίου": "South Aegean",
    "Βορείου Αιγαίου": "North Aegean",
}

# Map existing values in the `region` column to English
df["region"] = df["region"].replace(REGION_TRANSLATIONS)

In [152]:
df["region"].unique()

array(['Ionian Islands', 'Attica', 'Central Macedonia', 'Thessaly',
       'Crete', 'Central Greece', 'Epirus', 'Peloponnese',
       'Western Greece', 'Eastern Macedonia and Thrace',
       'Western Macedonia', 'South Aegean', 'North Aegean'], dtype=object)

In [153]:
df.head(3)

,project_code,size,incorporation_status,region,investment_budget,public_funding,nace_2_digits,nace_2_digits_description,sector_letter,sector_description
0,p_1,Small,Existing Business,Ionian Islands,482832.00,260504.28,11,Manufacture of beverages,C,Manufacturing
1,p_2,Medium,Existing Business,Attica,729388.06,255285.82,25,"Manufacture of fabricated metal products, exce...",C,Manufacturing
2,p_2,Medium,Existing Business,Attica,729388.06,255285.82,25,"Manufacture of fabricated metal products, exce...",C,Manufacturing


### 3.7 Save data to csv to make it publicly available

In [154]:
# enforce that nace_2_digits is saved as string
df["nace_2_digits"] = df["nace_2_digits"].astype(str)

In [155]:
df.to_csv("../data/smrt_manuf_projects_two_digit_activity.csv", index=False)